The below block of code imports the necessary libraries required for the project and also performs necessary operations on the turtles.

In [ ]:
import numpy as np
from numpy import inf
import random
import time
import networkx as nx             # Instructions for downloading the networkx library are provided in the README
import turtle                     # For visualization of the algorithm
from heapq import heappush, heappop      #Used in Bi-Directional Dijkstra
from itertools import count             #Used in Bi-Directional Dijkstra

In [1]:
dim = 12                   #dim is initialized as a global variable since it is used in almost all functions.
wn = turtle.Screen()
wn.bgcolor("white")        # set the window background color

subir = turtle.Turtle()    # Turtle for first run
ritwik = turtle.Turtle()   # Turtle for second run
ritwik.color("red")            
ritwik.pensize(3)
subir.color("blue")
subir.pensize(1)

The following function implements the bidirectional dijkstra algorithm.

In [ ]:
def bidirectional_dijkstra(G, source, target, weight=1):

    if source == target:
        return [source]
    push = heappush
    pop = heappop
    # Init:   Forward             Backward
    dists  = [{},                {}]  # dictionary of final distances
    paths  = [{source: [source]}, {target: [target]}]  # dictionary of paths
    fringe = [[],                []]  # heap of (distance, node) tuples for
                                      # extracting next node to expand
    seen   = [{source: 0},        {target: 0}]  # dictionary of distances to
                                                # nodes seen
    c = count()
    # initialize fringe heap
    push(fringe[0], (0, next(c), source))
    push(fringe[1], (0, next(c), target))
    # neighs for extracting correct neighbor information
    if G.is_directed():
        neighs = [G.successors_iter, G.predecessors_iter]
    else:
        neighs = [G.neighbors_iter, G.neighbors_iter]
    # variables to hold shortest discovered path
    #finaldist = 1e30000
    finalpath = []
    dir = 1
    while fringe[0] and fringe[1]:
        # choose direction
        # dir == 0 is forward direction and dir == 1 is back
        dir = 1 - dir
        # extract closest to expand
        (dist, _, v) = pop(fringe[dir])
        if v in dists[dir]:
            # Shortest path to v has already been found
            continue
        # update distance
        dists[dir][v] = dist  # equal to seen[dir][v]
        if v in dists[1 - dir]:
            # if we have scanned v in both directions we are done
            # we have now discovered the shortest path
            return finalpath

        for w in neighs[dir](v):
            if(dir == 0):  # forward
                if G.is_multigraph():
                    minweight = min((dd.get(weight, 1)
                                     for k, dd in G[v][w].items()))
                else:
                    minweight = G[v][w].get(weight, 1)
                vwLength = dists[dir][v] + minweight  # G[v][w].get(weight,1)
            else:  # back, must remember to change v,w->w,v
                if G.is_multigraph():
                    minweight = min((dd.get(weight, 1)
                                     for k, dd in G[w][v].items()))
                else:
                    minweight = G[w][v].get(weight, 1)
                vwLength = dists[dir][v] + minweight  # G[w][v].get(weight,1)

            if w in dists[dir]:
                if vwLength < dists[dir][w]:
                    raise ValueError(
                        "Contradictory paths found: negative weights?")
            elif w not in seen[dir] or vwLength < seen[dir][w]:
                # relaxing
                seen[dir][w] = vwLength
                push(fringe[dir], (vwLength, next(c), w))
                paths[dir][w] = paths[dir][v] + [w]
                if w in seen[0] and w in seen[1]:
                    # see if this path is better than than the already
                    # discovered shortest path
                    totaldist = seen[0][w] + seen[1][w]
                    if finalpath == [] or finaldist > totaldist:
                        finaldist = totaldist
                        revpath = paths[1][w][:]
                        revpath.reverse()
                        finalpath = paths[0][w] + revpath[1:]
    raise nx.NetworkXNoPath("No path between %s and %s." % (source, target))

A Stack class is made which will later be used for the backtracking algorithm discussed in the project report.

In [ ]:
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def peek(self):
        return self.items[len(self.items) - 1]

    def size(self):
        return len(self.items)

The heuristic function calculates the Manhattan distance between two points.

In [ ]:
def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

The steer function is used to steer the robot given the current heading of the robot and the desired rotation.

In [ ]:
def steer(heading, rotation):
    if rotation == 90:
        if heading == 'up':
            return 'right'
        if heading == 'down':
            return 'left'
        if heading == 'right':
            return 'down'
        if heading == 'left':
            return 'up'
    if rotation == -90:
        if heading == 'up':
            return 'left'
        if heading == 'down':
            return 'right'
        if heading == 'right':
            return 'up'
        if heading == 'left':
            return 'down'
    return heading

The move function moves the robot to a particular location given the current heading and distance to the location.

In [ ]:
def move(heading, location, dist):
    location = list(location)                  
    if heading == 'up':
        location[0] = location[0] + dist
    if heading == 'down':
        location[0] = location[0] - dist
    if heading == 'right':
        location[1] = location[1] + dist
    if heading == 'left':
        location[1] = location[1] - dist
    return location

The isGoal function is used to check whether the robot has reached the goal or not.

In [ ]:
def isGoal(location, dim):
    return (dim / 2 - 1) <= location[0] and location[0] <= (dim / 2) and (dim / 2 - 1) <= location[1] and location[
                                                                                                              1] <= (
                                                                                                              dim / 2)

THe hcost function is used to calculate the Hcost of the current location. 

In [ ]:
def hcost(location, dim):
    return np.amin([heuristic(location, [dim / 2, dim / 2]), heuristic(location, [dim / 2 - 1, dim / 2]),
                    heuristic(location, [dim / 2, dim / 2 - 1]), heuristic(location, [dim / 2 - 1, dim / 2 - 1])])

The traverse function returns the desired rotation and movement required to get from the current location to another location
given the current heading . The if loop deals with negative movements while the else loop deals with positive movements.

In [ ]:
def traverse(current_loc, desired_loc, heading):
    if (current_loc[1] - desired_loc[1] < 0 and heading == 'left') or (
                current_loc[1] - desired_loc[1] > 0 and heading == 'right') or (
                current_loc[0] - desired_loc[0] < 0 and heading == 'down') or (
                current_loc[0] - desired_loc[0] > 0 and heading == 'up'):
        if heading == 'left' or heading == 'right':
            return 0, -abs(current_loc[1] - desired_loc[1])
        else:
            return 0, -abs(current_loc[0] - desired_loc[0])
    else:
        if current_loc[0] == desired_loc[0]:
            movement = abs(current_loc[1] - desired_loc[1])
            if heading == 'left' or heading == 'right':
                rotation = 0
            elif heading == 'up':
                if current_loc[1] - desired_loc[1] < 0:
                    rotation = 90
                else:
                    rotation = -90
            else:
                if current_loc[1] - desired_loc[1] < 0:
                    rotation = -90
                else:
                    rotation = 90
        if current_loc[1] == desired_loc[1]:
            movement = abs(current_loc[0] - desired_loc[0])
            if heading == 'up' or heading == 'down':
                rotation = 0
            elif heading == 'left':
                if current_loc[0] - desired_loc[0] < 0:
                    rotation = 90
                else:
                    rotation = -90
            else:
                if current_loc[0] - desired_loc[0] < 0:
                    rotation = -90
                else:
                    rotation = 90
    return rotation, movement

The addtograph function takes in the current graph, current heading and location of the robot and the sensor input of the
current location. It then uses all of the above information to add edges between all detected cells which are adjacent to each
other. The graph stores nodes as a 1D value given by "Current Column Location *Dimension of Maze + Current Row Location"

In [ ]:
def addtograph(G, location, heading, sensors):
    loc = location[1] * dim + location[0]

    if heading == 'up':
        if sensors[0] > 0:
            for i in range(loc - dim, loc - dim * sensors[0] - 1, -dim):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[1] > 0:
            for i in range(loc + 1, loc + sensors[1] + 1, 1):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[2] > 0:
            for i in range(loc + dim, loc + dim * sensors[2] + 1, dim):
                G.add_edge(i, loc)
                loc = i
    elif heading == 'down':
        if sensors[0] > 0:
            for i in range(loc + dim, loc + dim * sensors[0] + 1, dim):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[1] > 0:
            for i in range(loc - 1, loc - sensors[1] - 1, -1):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[2] > 0:
            for i in range(loc - dim, loc - dim * sensors[2] - 1, -dim):
                G.add_edge(i, loc)
                loc = i
    elif heading == 'right':
        if sensors[0] > 0:
            for i in range(loc + 1, loc + sensors[0] + 1, 1):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[1] > 0:
            for i in range(loc + dim, loc + dim * sensors[1] + 1, dim):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[2] > 0:
            for i in range(loc - 1, loc - sensors[2] - 1, -1):
                G.add_edge(i, loc)
                loc = i
    else:
        if sensors[0] > 0:
            for i in range(loc - 1, loc - sensors[0] - 1, -1):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[1] > 0:
            for i in range(loc - dim, loc - dim * sensors[1] - 1, -dim):
                G.add_edge(i, loc)
                loc = i
        loc = location[1] * dim + location[0]
        if sensors[2] > 0:
            for i in range(loc + 1, loc + sensors[2] + 1, 1):
                G.add_edge(i, loc)
                loc = i
                

The cleanup_graph function further adds edges between nodes that are reachable from a particular location . For e.g. [3,0] might
be reachable from [0,0] and so is added . This ensures that the robot prefers taking straight paths over curves during the 
2nd run and also that robot takes more than 1 step whenever deemed necessary.

In [ ]:
def cleanup_graph(G):
    for node in G.nodes():
        for neighbor in G.neighbors(node):
            for child in G.neighbors(neighbor):
                if neighbor == node+1 and child == neighbor+1:
                    G.add_edge(node, child)
                    for grandchild in G.neighbors(child):
                        if grandchild == child+1:
                            G.add_edge(node, grandchild)
                if neighbor == node+dim and child == neighbor+dim:
                    G.add_edge(node, child)
                    for grandchild in G.neighbors(child):
                        if grandchild == child+dim:
                            G.add_edge(node, grandchild)

As earlier discussed the nodes in the graph are stored as a single value. This function is used to take that value and return
the actual coordinates for that value.

In [ ]:
def getcoord(loc):
    return loc % dim, loc / dim

The travel function takes 1D locations and the current heading and returns the rotation and movement required to get from the
start to the end.

In [ ]:
def travel(start, end, heading):
    current_loc = getcoord(start)
    desired_loc = getcoord(end)
    return traverse(current_loc, desired_loc, heading)

The following good neighbor function calculates the next best location of the neighbor using the visited counter matrix, current 
location of the robot and the current-known graph of the maze

In [ ]:
def goodneighbor(G, visited, location):
    currentloc = location[1] * dim + location[0]          #location as a 1D value as discussed earlier
    minimum = inf                                          #Initialize minimum visited count as infinity
    for neighbor in G.neighbors(currentloc):               #Iterate among neighbors of current node
        if minimum == visited[neighbor % dim][neighbor / dim]:
            if heurist_val > hcost([neighbor % dim, neighbor / dim], dim):  #Use hcost to resolve ties. Goal-Seeking Tendency
                minimum = visited[neighbor % dim][neighbor / dim]
                heurist_val = hcost([neighbor % dim, neighbor / dim], dim)
                optimumloc = neighbor
        if minimum > visited[neighbor % dim][neighbor / dim]:    #Use minimum visited count to select the best neighbour.
            minimum = visited[neighbor % dim][neighbor / dim]           #Update the new minimum visited count
            heurist_val = hcost([neighbor % dim, neighbor / dim], dim)  #Update heurist_val
            optimumloc = neighbor                                       #Set desired destination to this neighbour   
    return getcoord(optimumloc)                                         #Get Coordinates of desired destination neighbor

The following bad neighbor function calculates the next best location post-goal reach stint in the first run using the located 
counter matrix , current location and the current known graph of the maze.

In [ ]:
def badneighbor(G, located, location):
    currentloc = location[1] * dim + location[0]
    minimum = inf
    for neighbor in G.neighbors(currentloc):
        if minimum == located[neighbor % dim][neighbor / dim]:
            if heurist_val < hcost([neighbor % dim, neighbor / dim], dim): #Use Maximum Hcost to resolve ties.Goal-Repulsive 
                minimum = located[neighbor % dim][neighbor / dim]
                heurist_val = hcost([neighbor % dim, neighbor / dim], dim)
                optimumloc = neighbor
        if minimum > located[neighbor % dim][neighbor / dim]: #Uses located matrix inplace of visited matrix as in goodneighbor
            minimum = located[neighbor % dim][neighbor / dim] 
            heurist_val = hcost([neighbor % dim, neighbor / dim], dim)
            optimumloc = neighbor
    return getcoord(optimumloc)

The following function updates the located matrix . All cells which are located by the sensor once or more are given a value of
0.5(metric) while other visited cells are given a value of +1 which is incremented if they are visited more than once.

In [ ]:
def addtolocated(located, location, heading, sensors, metric):
    loc = location[1] * dim + location[0]

    if heading == 'up':
        if sensors[0] > 0:
            for i in range(loc - dim, loc - dim * sensors[0] - 1, -dim): #Iterate for all left cells
                x, y = getcoord(i)
                if located[x][y] == 0:                         #If location was previously neither visited nor located
                    located[x][y] += metric                      # Add 0.5 to the current located coordinate 
        if sensors[1] > 0:
            for i in range(loc + 1, loc + sensors[1] + 1, 1):       #iterate for all forward cells
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
        if sensors[2] > 0:                                           
            for i in range(loc + dim, loc + dim * sensors[2] + 1, dim):   #Iterate for all right cells.
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
    elif heading == 'down':
        if sensors[0] > 0:
            for i in range(loc + dim, loc + dim * sensors[0] + 1, dim):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
        if sensors[1] > 0:
            for i in range(loc - 1, loc - sensors[1] - 1, -1):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
        if sensors[2] > 0:
            for i in range(loc - dim, loc - dim * sensors[2] - 1, -dim):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
    elif heading == 'right':
        if sensors[0] > 0:
            for i in range(loc + 1, loc + sensors[0] + 1, 1):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
        if sensors[1] > 0:
            for i in range(loc + dim, loc + dim * sensors[1] + 1, dim):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
        if sensors[2] > 0:
            for i in range(loc - 1, loc - sensors[2] - 1, -1):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
    else:
        if sensors[0] > 0:
            for i in range(loc - 1, loc - sensors[0] - 1, -1):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
        if sensors[1] > 0:
            for i in range(loc - dim, loc - dim * sensors[1] - 1, -dim):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric
        if sensors[2] > 0:
            for i in range(loc + 1, loc + sensors[2] + 1, 1):
                x, y = getcoord(i)
                if located[x][y] == 0:
                    located[x][y] += metric

This function is used to calculate coverage of the maze given the located matrix as calculated earlier. Total Coverage is 
incremented by 1 if the cell has been visited once or more and is incremented by 0.5 if the cell has been located but not visited.
Then Coverage= 100 X TotalCoverage/(Dim*Dim)

In [ ]:
def showcoverage(located):
    coverage = 0
    for i in range(dim):
        for j in range(dim):
            if located[i][j] == 0.5:     #Check if location has been located by the sensor but not visited
                coverage += 0.5
            if located[i][j] >= 1:       #Check if location has been visited atleast once.
                coverage += 1
    return (coverage*100.0)/(dim*dim)

The backtracker function is a major chunk of the implementation of the backtracker algorithm. It takes in the stack s, current
heading of the robot , the vis2 matrix which has values 0 for non visited cells, values -1 for cells from whom backtracking was
done and value 1 for all other cells.

In [ ]:
def backtracker(s, heading, vis2, G):
    currentloc = s.peek()             #Current location is always pushed at the top of the stack
    vis2[currentloc] = 1              #Current location always has value 1
    heurist_val = inf                 
    flag = 0                   
    for neighbor in G.neighbors(currentloc):
        if vis2[neighbor] == 0:            #Check for unvisited cells
            flag = 1                       # Put flag = 1 so that visited cells are ignored in the further code for the location.
            if heurist_val > hcost([neighbor % dim, neighbor / dim], dim):
                desiredloc = neighbor       #Use hcost to resolve ties
                heurist_val = hcost([neighbor % dim, neighbor / dim], dim)
    for neighbor in G.neighbors(currentloc):
        if vis2[neighbor] == 1 and flag == 0:   #If no unvisited cells
            flag = 1                            #Put flag = 1 so that other cells are now ignored.
            s.pop()                             # Pop the current location from the stack
            desiredloc = s.peek()               # Desired location is the top element of the stack--> Backtracking till unvisited
                                                #cells are found.
            vis2[currentloc] = -1               #Set the popped out location value to -1
    print vis2                                  
    return travel(currentloc, desiredloc, heading) #Call travel function to return required rotation, movement.

The badbacktracker function is the post -goal reach stint version of the backtracker algorithm. The only difference is that it
uses maximum hcost to resolve ties. Everything else is the same. 

In [ ]:
def badbacktracker(s, heading, vis2, G):
    currentloc = s.peek()
    vis2[currentloc] = 1
    heurist_val = -inf               #Set heurist_val to -inf since we are taking a goal repulsive tendency.
    flag = 0
    for neighbor in G.neighbors(currentloc):
        if vis2[neighbor] == 0:
            flag = 1
            if heurist_val < hcost([neighbor % dim, neighbor / dim], dim):
                desiredloc = neighbor
                heurist_val = hcost([neighbor % dim, neighbor / dim], dim)
    for neighbor in G.neighbors(currentloc):
        if vis2[neighbor] == 1 and flag == 0:
            flag = 1
            s.pop()
            desiredloc = s.peek()
            vis2[currentloc] = -1
    print vis2
    return travel(currentloc, desiredloc, heading)

The locbacktracker is the post-goal reach stint version of the backtracker algorithm. As opposed to the Bad BackTracker Algorithm
it uses the minimum located counter to resolve ties and the maximum heuristic value to further resolve ties.

In [ ]:
def locbacktracker(s, heading, vis2, G, located):
    currentloc = s.peek()
    vis2[currentloc] = 1
    heurist_val = -inf
    flag = 0
    minimum = inf
    for neighbor in G.neighbors(currentloc):
        if vis2[neighbor] == 0:
            flag = 1
            if minimum == located[neighbor % dim][neighbor / dim]:
                if heurist_val < hcost([neighbor % dim, neighbor / dim], dim):
                    minimum = located[neighbor % dim][neighbor / dim]
                    heurist_val = hcost([neighbor % dim, neighbor / dim], dim)
                    desiredloc = neighbor
            if minimum > located[neighbor % dim][neighbor / dim]:
                minimum = located[neighbor % dim][neighbor / dim]
                heurist_val = hcost([neighbor % dim, neighbor / dim], dim)
                desiredloc = neighbor
    for neighbor in G.neighbors(currentloc):
        if vis2[neighbor] == 1 and flag == 0:
            flag = 1
            s.pop()
            desiredloc = s.peek()
            vis2[currentloc] = -1
    print vis2
    return travel(currentloc, desiredloc, heading)

The robot class is now discussed . Comments are included at appropriate places. 

In [ ]:
class Robot(object):
    def __init__(self, maze_dim):
        self.location = [0, 0]   #Initial location
        self.heading = 'up'      #Initial heading
        self.maze_dim = maze_dim
        global dim
        dim = maze_dim           # Set the global variable dim
        self.flag = 0            # Used for goal reached or not in the first run
        self.visited = np.zeros((self.maze_dim, self.maze_dim))  #Initialize visited matrix to zeros
        self.located = np.zeros((self.maze_dim, self.maze_dim))  #Initialize located matrix to zeros
        self.vis2 = np.zeros(self.maze_dim*self.maze_dim)        #Initialize vis2 matrix to zeros
        self.visited[0][0] = 1                                   
        self.located[0][0] = 1
        self.goal_loc = [0, 0]                                  #Initialize goal location
        self.count = 1                                          
        G = nx.Graph()                                          # Create a graph
        for i in range(0, dim * dim):
            G.add_node(i)                                       # Add all dim*dim nodes to it. 
        self.graph = G  
        self.path = []      #Initialize optimal shortest path (2nd Run)                
        self.gn1 = 1        # 1-> Implements Good Neighbor Algorithm for Pre-GoalReach Stint
                            # 0-> Implements Backtracking Algorithm for Pre-GoalReach Stint
        self.gn2 = 0        # 1-> Implements Good Neighbor Algorithm for Post-GoalReach Stint
                            # 0-> Implements Backtracking Algorithm for Post-GoalReach Stint
        self.s = Stack()    #Create a stack
        #Put both turtles to start position and set their heading
        subir.penup()
        subir.setpos(-10*dim+10, -10*dim+10)
        subir.pendown()
        subir.left(90)
        ritwik.penup()
        ritwik.setpos(-10 * dim + 10, -10 * dim + 10)
        ritwik.pendown()
        ritwik.left(90)                                         
        #Subir -> First Run turtle : Ritwik -> Second Run Turtle

    def next_move(self, sensors):
        if self.flag == 0 and self.gn1 == 1:    #Implementing Good Neighbor Algorithm for the pre-goal reach stint (1st Run)
            addtograph(self.graph, self.location, self.heading, sensors)  #Updating Graph
            cleanup_graph(self.graph)                                     #Updating Graph
            addtolocated(self.located, self.location, self.heading, sensors, 0.5)   #Updating located matrix            
            rotation, movement = traverse(self.location, goodneighbor(self.graph, self.visited, self.location),
                                          self.heading)                #Calling the good neighbor algorithm for required
                                                                       #rotation and movement
            if np.amax(sensors) == 0 or movement == -1:                  #If deadend or moving backwards 
                self.visited[self.location[0]][self.location[1]] += 1   #Increment counter for visited matrix
                self.located[self.location[0]][self.location[1]] += 1   #Increment counter for located matrix
            subir.right(rotation)                                       #Rotate the turtle
            subir.forward(movement*20)                                  #Move the turtle
            self.heading = steer(self.heading, rotation)                #Calculate new heading to be used as current heading
                                                                        #for the next time step
            self.location = move(self.heading, self.location, movement) #Calculate new location to be used as current location
                                                                        #for the next time step
            self.visited[self.location[0]][self.location[1]] += 1       #Increment counter for visited matrix for the going to 
                                                                        #be new location
            self.located[self.location[0]][self.location[1]] += 1       #Increment counter for located matrix for the going to
                                                                        #be new location

            if isGoal(self.location, self.maze_dim):                 #Check if goal is reached
                self.flag = 1
                self.goal_loc = self.location                        #Set goal location 
                self.count = 0                                      
                time.sleep(0.5)                                      #Take rest :P
                print "GOAL IS REACHED YOHOO!!!"                      
                time.sleep(1)                                        #Rejoice for 1 second! :P.
        elif self.flag == 0 and self.gn1 == 0:  #Implementing BackTracking Algorithm for the pre-goal reach stint (1st Run)
            addtograph(self.graph, self.location, self.heading, sensors)
            cleanup_graph(self.graph)
            addtolocated(self.located, self.location, self.heading, sensors, 0.5)
            if self.vis2[self.location[1] * dim + self.location[0]] == 0:   #If location not visited before
                self.vis2[self.location[1] * dim + self.location[0]] = 1      #Update vis2[current_location] to 1
                self.s.push(self.location[1] * dim + self.location[0])        #Push it into the stack
            rotation, movement = backtracker(self.s, self.heading, self.vis2, self.graph)  #Get Desired Rotation and Movement.
            if np.amax(sensors) == 0 or movement < 0:
                self.located[self.location[0]][self.location[1]] += 1
            subir.right(rotation)
            subir.forward(movement * 20)
            self.heading = steer(self.heading, rotation)
            self.location = move(self.heading, self.location, movement)
            self.located[self.location[0]][self.location[1]] += 1

            if isGoal(self.location, self.maze_dim):
                self.flag = 1
                self.goal_loc = self.location
                self.count = 0
                time.sleep(0.5)
                print "GOAL IS REACHED YOHOO!!!"
                time.sleep(1)
            #Now the PRE-GOAL REACH Stint comes to an END. We will now explore atleast 75% of the maze if not already done.
            
        elif self.flag == 1 and self.count == 0 and showcoverage(self.located) < 76 and self.gn2 == 1: #Bad Neighbour Algo
            addtograph(self.graph, self.location, self.heading, sensors)
            cleanup_graph(self.graph)
            addtolocated(self.located, self.location, self.heading, sensors, 0.5)
            rotation, movement = traverse(self.location,
                                          badneighbor(self.graph, self.located, self.location), self.heading)
            if np.amax(sensors) == 0 or movement == -1:
                self.located[self.location[0]][self.location[1]] += 1
            subir.right(rotation)
            subir.forward(movement * 20)
            self.heading = steer(self.heading, rotation)
            self.location = move(self.heading, self.location, movement)
            self.located[self.location[0]][self.location[1]] += 1
            
        elif self.flag == 1 and self.count == 0 and showcoverage(self.located) < 76 and self.gn2 == 0:#Location Backtracker Algo
            addtograph(self.graph, self.location, self.heading, sensors)
            cleanup_graph(self.graph)
            addtolocated(self.located, self.location, self.heading, sensors, 0.5)
            if self.vis2[self.location[1] * dim + self.location[0]] == 0:
                self.vis2[self.location[1] * dim + self.location[0]] = 1
                self.s.push(self.location[1] * dim + self.location[0])
            rotation, movement = locbacktracker(self.s, self.heading, self.vis2, self.graph, self.located) 
            #Change func to badbacktracker(self.s, self.heading, self.vis2, self.graph) to visualize the results of the 
            #Bad BackTracker Algorithm
            if np.amax(sensors) == 0 or movement < 0:
                self.located[self.location[0]][self.location[1]] += 1
            subir.right(rotation)
            subir.forward(movement * 20)
            self.heading = steer(self.heading, rotation)
            self.location = move(self.heading, self.location, movement)
            self.located[self.location[0]][self.location[1]] += 1
            #We now have completed the First Run with both Pre- and Post- Goal Reach Stints. We move on to the 2nd Run.
        else:
            if self.count == 0:        #If Reset is not returned for start for Second Run Yet.
                self.count = 1
                cleanup_graph(self.graph)
                self.path = nx.shortest_path(self.graph, 0, self.goal_loc[1] * dim + self.goal_loc[0]) #Find the shortest path
                                                                                                    #Store it into self.path
                print len(self.path)  #See the no. of moves it will take us.
                self.location = [0, 0]          
                self.heading = 'up'
                return 'Reset', 'Reset'       #Return Reset, Reset to start the second run
            else:
                rotation, movement = travel(self.location[1] * dim + self.location[0], self.path[self.count],
                                            self.heading)
                ritwik.right(rotation)
                ritwik.forward(movement * 20)
                self.heading = steer(self.heading, rotation)
                self.location = move(self.heading, self.location, movement)
                self.count += 1
        return rotation, movement